In [1]:
from sparkdl import readImages
from pyspark.sql.functions import lit

Using TensorFlow backend.


In [2]:

img_dir = "/home/nkamat/Pictures/personalities/"

#Read images and Create training & test DataFrames for transfer learning
jobs_df = readImages(img_dir + "/jobs").withColumn("label", lit(1))
zuckerberg_df = readImages(img_dir + "/zuckerberg").withColumn("label", lit(0))
jobs_train, jobs_test = jobs_df.randomSplit([0.6, 0.4])
zuckerberg_train, zuckerberg_test = zuckerberg_df.randomSplit([0.6, 0.4])

#dataframe for training a classification model
train_df = jobs_train.unionAll(zuckerberg_train)

In [3]:
test_df = jobs_test.unionAll(zuckerberg_test)

In [4]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])
p_model = p.fit(train_df)

87474176/87910968 [============================>.] - ETA: 0sINFO:tensorflow:Froze 376 variables.
Converted 376 variables to const ops.
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.


In [5]:
predictions = p_model.transform(test_df)

predictions.select("filePath", "prediction").show(truncate=False)

INFO:tensorflow:Froze 376 variables.
Converted 376 variables to const ops.
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
+--------------------------------------------------------------+----------+
|filePath                                                      |prediction|
+--------------------------------------------------------------+----------+
|file:/home/nkamat/Pictures/personalities/jobs/steve4.jpg      |0.0       |
|file:/home/nkamat/Pictures/personalities/jobs/steve5.jpg      |1.0       |
|file:/home/nkamat/Pictures/personalities/jobs/steve7.jpg      |1.0       |
|file:/home/nkamat/Pictures/personalities/jobs/steve9.jpg      |1.0       |
|file:/home/nkamat/Pictures/personalities/jobs/steve10.jpg     |1.0       |
|file:/home/nkamat/Pictures/personalities/jobs/steve12.jpg     |1.0       |
|file:/home/nkamat/Pictures/personalities/jobs/steve6.jpg      |1.0       |
|file:/home/nkamat/Pictures/personalities/zuckerberg/mark2.jpg |0.0       |
|file:/home/nkamat